First we want to clean the data of columns we wont use/need.
Next get rid of all the hard to work with data in this dataset. This is done using integer codes found in the CSV. They describe the date and precision of coordinates

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
from GooseUtils import get_coord_unc

from IPython.display import HTML, Image


#Skipping bad lines as we only lose 5 columns out of 6mil entries- bad line as a result of an extra column
"""https://www.sciencebase.gov/catalog/file/get/653fa806d34ee4b6e05bc57d?f=__disk__3e%2F93%2F14%2F3e9314e6fc046735f91cc3e78a9982b5f72d8c59"""
canada_geese_df = pd.read_csv("Bird_Banding_Data/NABBP_2023_grp_02.csv", on_bad_lines='skip') 

"""https://www.sciencebase.gov/catalog/file/get/653fa806d34ee4b6e05bc57d?f=__disk__a5%2Faa%2F46%2Fa5aa46b4bba48273e9ab437e1ee5f7eed5a3aa3f"""
subspecies_df = pd.read_csv("Bird_Banding_Data/NABBP_2023_grp_03.csv", on_bad_lines='skip')

subspecies_df = subspecies_df[(subspecies_df['SPECIES_ID'] == 1729) | (subspecies_df['SPECIES_ID'] == 1723)]

canada_geese_df = pd.concat([canada_geese_df, subspecies_df], ignore_index=True)

#Drop all columns that we most likely wont use in the future
canada_geese_df = canada_geese_df.drop(columns=['ISO_COUNTRY', 'ISO_SUBDIVISION', 'AGE_CODE', 'SEX_CODE', 'EXTRA_INFO',
 'HOW_OBTAINED', 'WHO_OBTAINED','REPORTING_METHOD', 'PRESENT_CONDITION', 'MIN_AGE_AT_ENC', 'PERMIT', 'RECORD_SOURCE', 
 'BAND_STATUS','BIRD_STATUS', 'BAND_TYPE'])


#Drop all Rows with coords in country / hard to work with percision codes
canada_geese_df = canada_geese_df[canada_geese_df['COORD_PREC'] != 72]
canada_geese_df = canada_geese_df[canada_geese_df['COORD_PREC'] != 33]
canada_geese_df = canada_geese_df[canada_geese_df['COORD_PREC'] != 38]
canada_geese_df = canada_geese_df[canada_geese_df['COORD_PREC'] != 28]
canada_geese_df = canada_geese_df[canada_geese_df['COORD_PREC'] != 18]
canada_geese_df = canada_geese_df[canada_geese_df['COORD_PREC'] != 8]
canada_geese_df = canada_geese_df[canada_geese_df['COORD_PREC'] != 12]
canada_geese_df = canada_geese_df[canada_geese_df['COORD_PREC'].notna()]

#Droping rows with missing months and years
canada_geese_df = canada_geese_df[(canada_geese_df['EVENT_MONTH'] < 13) & (canada_geese_df['EVENT_MONTH'] > 0)]
canada_geese_df = canada_geese_df[(canada_geese_df['EVENT_DAY'] < 32) & (canada_geese_df['EVENT_MONTH'] > 0)]

canada_geese_df['EVENT_DATE'] = pd.to_datetime(canada_geese_df['EVENT_DATE'], errors='coerce')
canada_geese_df = canada_geese_df[canada_geese_df['EVENT_DATE'].notna()]

canada_geese_df['COORD_UNC'] = canada_geese_df['COORD_PREC'].apply(get_coord_unc)

display(canada_geese_df)

C:\Users\Boche.DESKTOP-3RJ9K3E\AppData\Local\Temp\ipykernel_2784\214924327.py:10: DtypeWarning: Columns (25,27) have mixed types. Specify dtype option on import or set low_memory=False.
  canada_geese_df = pd.read_csv("Bird_Banding_Data/NABBP_2023_grp_02.csv", on_bad_lines='skip')
C:\Users\Boche.DESKTOP-3RJ9K3E\AppData\Local\Temp\ipykernel_2784\214924327.py:13: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  subspecies_df = pd.read_csv("Bird_Banding_Data/NABBP_2023_grp_03.csv", on_bad_lines='skip')


,RECORD_ID,EVENT_TYPE,BAND,ORIGINAL_BAND,OTHER_BANDS,EVENT_DATE,EVENT_DAY,EVENT_MONTH,EVENT_YEAR,SPECIES_ID,SPECIES_NAME,LAT_DD,LON_DD,COORD_PREC,COORD_UNC
0,-42492898,B,B19439751829,B19439751829,NaN,1960-07-04,4,7,1960,1720,Canada Goose,50.50000,-112.50000,60.0,0.500000
1,-89657255,B,B59439751813,B59439751813,NaN,1960-07-04,4,7,1960,1720,Canada Goose,50.50000,-112.50000,60.0,0.500000
2,-42492897,B,B79439751883,B79439751883,NaN,1960-07-04,4,7,1960,1720,Canada Goose,50.50000,-112.50000,60.0,0.500000
3,-89657254,B,B59439751734,B59439751734,NaN,1960-07-04,4,7,1960,1720,Canada Goose,50.50000,-112.50000,60.0,0.500000
4,-41594736,B,B99439751845,B99439751845,NaN,1960-07-04,4,7,1960,1720,Canada Goose,50.50000,-112.50000,60.0,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5994888,-9905925,E,B98475892054,B98475892054,NaN,2016-11-28,28,11,2016,1729,Canada Goose,41.90028,-104.34956,11.0,0.250000
5994889,-12192357,E,B58825645102,B58825645102,NaN,2017-02-12,12,2,2017,1729,Canada goose,42.21274,-104.51718,11.0,0.250000
5994890,-12180189,E,B88215024872,B88215024872,NaN,2018-02-10,10,2,2018,1729,Small Canada Goose,41.86380,-104.12930,0.0,0.000005
5994891,-12162813,E,B87865462176,B87865462176,NaN,2018-12-11,11,12,2018,1729,Cackling Goose,42.13635,-104.42337,11.0,0.250000


Central therom find group by band IDS 

In [ ]:
sampled_geese = canada_geese_df.sample(n=250000, random_state=33) #33 cause it's my favorite number

plt.figure(figsize=(12,6))
plt.scatter(sampled_geese['EVENT_DATE'], sampled_geese['LAT_DD'],)
